In [1]:
import os
import pandas as pd
import json
import tqdm
import numpy as np
#
from multiprocessing import Pool
from threading import Thread
from multiprocessing import Process
from functools import reduce
#
from js_flat_tools import df_slicer,FLAT_JS

In [9]:
import datetime

In [12]:
datetime.datetime.now()

datetime.datetime(2022, 5, 8, 11, 11, 9, 587744)

In [3]:
def Load_War(year,war_id,folder_path = 'E:\Data\Eve-Online\Wars'):
    df = pd.DataFrame()
    idx = 0
    JS_paths =  os.listdir(folder_path+f'\\wars-{year}\\wars\\{war_id}\\killmails')
    for idx in tqdm.trange(len(JS_paths)):
        file_path = folder_path+f'\\wars-2016\\wars\\{war_id}\\killmails\\' + JS_paths[idx]
        #print(file_path)
        with open(file_path) as file:
            try:
                J = json.load(file)
                df = pd.concat([df,pd.json_normalize(J)])
            except:
                pass 
        idx +=1 
    df.to_csv(f'E:\\Data\\Eve-Online\\Wars_Csv\\{year}war_{war_id}.csv')

In [6]:
def Load_Raid(Path_list,year):
    df = pd.DataFrame()
    idx = 0
    for path in tqdm.tqdm(Path_list):
        file_path = f"E:\Data\Eve-Online\Wars\wars-{year}\wars\{path}"
        with open(file_path) as file:
            try:
                J = json.load(file)
                df = pd.concat([df,pd.json_normalize(J)])
            except:
                pass 
        idx +=1 
    df.to_csv(f'E:\\Data\\Eve-Online\\Wars_Csv\\{year}war_RAID.csv')

In [14]:
#Load Wars
for year in [2017,2018,2019]:
    Existing_Wars = [ i.split('.')[0].split('_')[1] for i in os.listdir(f'E:\Data\Eve-Online\Wars_Csv')]
    Wars_ = [ i for i in os.listdir(f'E:\Data\Eve-Online\Wars\wars-{year}\wars') if '.json' not in i]
    Wars = [ i for i in Wars_ if i not in Existing_Wars]
    for war_id in tqdm.tqdm(Wars):
        try:
            Load_War(year,war_id)
        except FileNotFoundError as e:
            print(e)

100%|██████████| 8866/8866 [07:42<00:00, 19.18it/s]   


In [7]:
#Load Raids
for year in [2016,2017,2018,2019]:
    #Existing_Wars = [ i.split('.')[0].split('_')[1] for i in os.listdir(f'E:\Data\Eve-Online\Wars_Csv')]
    Wars_ = [ i for i in os.listdir(f'E:\Data\Eve-Online\Wars\wars-{year}\wars') if '.json' in i]
    #Wars = [ i for i in Wars_ if i not in Existing_Wars]
    Load_Raid(Wars_,year)

100%|██████████| 36093/36093 [02:17<00:00, 262.23it/s]


### Aggregate Files with same year

In [13]:
for year in [2016,2017,2018,2019]:
    Wars_ = [ i for i in os.listdir(f'E:\Data\Eve-Online\Wars_Csv') if str(year)+'war' in i]
    out = pd.DataFrame()
    for file in tqdm.tqdm(Wars_):
        file_path = f'E:\Data\Eve-Online\Wars_Csv\{file}'
        df_ = pd.read_csv(file_path,index_col=0)
        out = pd.concat([out,df_])
    out.to_csv(f'E:\Data\Eve-Online\{year}_wars.csv' ,index =False)

100%|██████████| 8867/8867 [03:50<00:00, 38.49it/s] 


In [15]:
out.columns

Index(['attackers', 'killmail_id', 'killmail_time', 'solar_system_id',
       'war_id', 'killmail_hash', 'http_last_modified', 'victim.character_id',
       'victim.corporation_id', 'victim.damage_taken', 'victim.items',
       'victim.position.x', 'victim.position.y', 'victim.position.z',
       'victim.ship_type_id', 'victim.alliance_id', 'moon_id',
       'victim.faction_id', 'allies', 'declared', 'finished', 'id', 'mutual',
       'open_for_allies', 'retracted', 'started', 'aggressor.corporation_id',
       'aggressor.isk_destroyed', 'aggressor.ships_killed',
       'defender.corporation_id', 'defender.isk_destroyed',
       'defender.ships_killed', 'defender.alliance_id',
       'aggressor.alliance_id'],
      dtype='object')

In [18]:
out[['allies', 'declared', 'finished', 'id', 'mutual',
       'open_for_allies', 'retracted', 'started', 'aggressor.corporation_id',
       'aggressor.isk_destroyed', 'aggressor.ships_killed',
       'defender.corporation_id', 'defender.isk_destroyed',
       'defender.ships_killed', 'defender.alliance_id',
       'aggressor.alliance_id']]

,allies,declared,finished,id,mutual,open_for_allies,retracted,started,aggressor.corporation_id,aggressor.isk_destroyed,aggressor.ships_killed,defender.corporation_id,defender.isk_destroyed,defender.ships_killed,defender.alliance_id,aggressor.alliance_id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,[],2009-11-27T22:05:00Z,2019-02-13T12:02:00Z,96623.0,True,False,2019-02-12T12:02:00Z,2009-11-28T22:05:00Z,1.780746e+09,0.0,0.0,1.781631e+09,0.0,0.0,NaN,NaN
0,[],2009-11-29T13:41:00Z,2019-02-13T11:50:00Z,96767.0,True,False,2019-02-12T11:50:00Z,2009-11-30T13:41:00Z,1.815164e+09,0.0,0.0,1.513802e+09,0.0,0.0,NaN,NaN
0,[],2009-11-29T13:46:00Z,2019-02-13T11:50:00Z,96768.0,True,False,2019-02-12T11:50:00Z,2009-11-30T13:46:00Z,1.816250e+09,0.0,0.0,1.513802e+09,0.0,0.0,NaN,NaN
0,[],2009-12-17T09:24:00Z,2019-02-13T11:50:00Z,98308.0,True,False,2019-02-12T11:50:00Z,2009-12-18T09:24:00Z,1.519212e+09,0.0,0.0,1.517408e+09,0.0,0.0,NaN,NaN


In [37]:
type_a = out[out['attackers'].apply(lambda x: str(x) != 'nan')]
type_a.isna().sum()/type_a.shape[0]

attackers                   0.000000
killmail_id                 0.000000
killmail_time               0.000000
solar_system_id             0.000000
war_id                      0.000000
killmail_hash               0.000000
http_last_modified          0.000000
victim.character_id         0.057336
victim.corporation_id       0.000000
victim.damage_taken         0.000000
victim.items                0.000000
victim.position.x           0.000000
victim.position.y           0.000000
victim.position.z           0.000000
victim.ship_type_id         0.000000
victim.alliance_id          0.090346
moon_id                     0.989918
victim.faction_id           0.992909
allies                      1.000000
declared                    1.000000
finished                    1.000000
id                          1.000000
mutual                      1.000000
open_for_allies             1.000000
retracted                   1.000000
started                     1.000000
aggressor.corporation_id    1.000000
a

In [36]:
type_b = out[out['attackers'].apply(lambda x: str(x) == 'nan')]
type_b.isna().sum()/type_b.shape[0]

attackers                   1.000000
killmail_id                 1.000000
killmail_time               1.000000
solar_system_id             1.000000
war_id                      0.000000
killmail_hash               1.000000
http_last_modified          0.000000
victim.character_id         1.000000
victim.corporation_id       1.000000
victim.damage_taken         1.000000
victim.items                1.000000
victim.position.x           1.000000
victim.position.y           1.000000
victim.position.z           1.000000
victim.ship_type_id         1.000000
victim.alliance_id          1.000000
moon_id                     1.000000
victim.faction_id           1.000000
allies                      0.000000
declared                    0.000000
finished                    0.000028
id                          0.000000
mutual                      0.000000
open_for_allies             0.000000
retracted                   0.000028
started                     0.000000
aggressor.corporation_id    0.393123
a

### Conclusion
After exporing the data composition, it seems that 2 types of data were mixed. 

In [38]:
df_all = pd.DataFrame()
for year in [2016,2017,2018,2019]:
    df_ = pd.read_csv(f'E:\Data\Eve-Online\{year}_wars.csv')
    df_all = pd.concat([df_all,df_])

C:\Users\18066\AppData\Local\Temp\ipykernel_36904\4135360281.py:3: DtypeWarning: Columns (0,2,5,11,18,19,20,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ = pd.read_csv(f'E:\Data\Eve-Online\{year}_wars.csv')
C:\Users\18066\AppData\Local\Temp\ipykernel_36904\4135360281.py:3: DtypeWarning: Columns (0,2,5,10,18,19,20,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ = pd.read_csv(f'E:\Data\Eve-Online\{year}_wars.csv')
C:\Users\18066\AppData\Local\Temp\ipykernel_36904\4135360281.py:3: DtypeWarning: Columns (0,2,5,10,18,19,20,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ = pd.read_csv(f'E:\Data\Eve-Online\{year}_wars.csv')
C:\Users\18066\AppData\Local\Temp\ipykernel_36904\4135360281.py:3: DtypeWarning: Columns (0,2,5,10,18,19,20,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ = pd.read_csv(f'E:\Data\Eve-Online\{y

In [42]:
type_a = df_all[df_all['attackers'].apply(lambda x: str(x) != 'nan')]

In [50]:
temp = type_a.isna().sum()/type_a.shape[0]
temp

attackers                   0.000000
killmail_id                 0.000000
killmail_time               0.000000
solar_system_id             0.000000
war_id                      0.000000
killmail_hash               0.000000
http_last_modified          0.000000
victim.alliance_id          0.219212
victim.character_id         0.078330
victim.corporation_id       0.000000
victim.damage_taken         0.000000
victim.items                0.000000
victim.position.x           0.000000
victim.position.y           0.000000
victim.position.z           0.000000
victim.ship_type_id         0.000000
moon_id                     0.958051
victim.faction_id           0.974436
allies                      1.000000
declared                    1.000000
finished                    1.000000
id                          1.000000
mutual                      1.000000
open_for_allies             1.000000
retracted                   1.000000
started                     1.000000
aggressor.corporation_id    1.000000
a

In [51]:
feature_keep = list(temp[temp == 0].keys()) + ['victim.alliance_id','victim.character_id']

In [56]:
df_wars = type_a[feature_keep]

In [84]:
df_attackerinfo = pd.DataFrame()
for idx in tqdm.trange(df_wars.shape[0]):
    row = df_wars.iloc[idx]
    JS_str = row['attackers']
    kill_mail_id = row['killmail_id']
    js_dct_lst = eval(JS_str)
    for dct in js_dct_lst:
        dct['killmail_id'] = kill_mail_id
    df_temp = pd.json_normalize(js_dct_lst)
    df_attackerinfo = pd.concat([df_attackerinfo,df_temp])

100%|██████████| 837851/837851 [11:32:36<00:00, 20.16it/s]  


In [18]:
df_attackerinfo.to_csv('E:\\Data\\Eve-Online\\attackers.csv',index=False)

In [45]:
#df_wars.to_csv('E:\\Data\\Eve-Online\\WARS.csv')
df_wars = pd.read_csv('E:\Data\Eve-Online\WARS.csv',index_col=0)

In [112]:
df_attackerinfo.groupby('corporation_id').count().sort_values('character_id',ascending=False).head(10)

,alliance_id,character_id,damage_done,final_blow,security_status,ship_type_id,weapon_type_id,killmail_id,faction_id
corporation_id,,,,,,,,,
9.838831e+07,349361,348949,349633,349633,349633,345436,348949,349633,0
9.855851e+07,264745,264741,264745,264745,264745,262652,264741,264745,0
3.544952e+08,179871,179698,179880,179880,179880,177262,179698,179880,0
9.837086e+07,102531,102530,102535,102535,102535,100860,102530,102535,0
9.812351e+07,69725,69725,69725,69725,69725,68850,69725,69725,0
9.847777e+07,57937,57934,57953,57953,57953,57382,57934,57953,0
1.018390e+09,57785,57785,57789,57789,57789,56089,57785,57789,0
9.809584e+07,38734,55148,55148,55148,55148,54517,55148,55148,0
1.741771e+09,48099,48099,48125,48125,48125,46507,48099,48125,39


In [ ]:
out_df = pd.DataFrame()
for year in [2017,2018,2019]:
    path = f'E:\Data\Eve-Online\{year}war_RAID.csv'
    df_ = pd.read_csv(path,index_col=0)
    out_df = pd.concat([df_,out_df])
    
out_df.to_csv('E:\Data\Eve-Online\ENCOUNTER.csv')

In [9]:
_df = pd.read_csv('E:\Data\Eve-Online\ENCOUNTER.csv',index_col=0)
_df = _df.drop(columns=['allies'])

In [12]:
_df.reset_index(drop=True).to_csv('E:\Data\Eve-Online\ENCOUNTER.csv',index=False)

### Items

In [25]:

def df_slicer(df,nslice):
    """_summary_
    divides a data frame into n equally distributed slices 
    Args:
        df (pd.DataFrame): the data frame to be sliced 
        nslice (int): number of slices to be divied into 

    Returns:
        [*pd.DataFrame]: a list of dataframes seperated into n slices
    """
    from math import ceil
    n_rows = df.shape[0]
    scope = ceil(n_rows/nslice)
    out = []
    head = 0
    tail = head + scope 
    while True:
        #print(f'{head}->{tail}')
        out.append(df[head:tail])
        head = tail 
        tail = head + scope 
        if len(out) ==  nslice:
            break
    return out 

def set_ids(dct,killmail_id,victim_character_id):
    dct['killmail_id'] = killmail_id
    dct['victim.character_id'] = victim_character_id
    return dct 

def flatten_js(item_sample,killmail_id,victim_character_id):
    return pd.json_normalize(list(map(lambda x: set_ids(x,killmail_id,victim_character_id),eval(item_sample))))

def FLAT_JS(df,hash,out_dct = None,target_column = 'victim.items'):
    #out_df = pd.DataFrame()
    for idx in tqdm.trange(df.shape[0]):
        row = df.iloc[idx]
        JS = row[target_column]
        killmail_id = row['killmail_id']
        victim_character_id = row['victim.character_id']
        df_temp = flatten_js(JS,killmail_id,victim_character_id)
        #out_df = pd.concat([out_df,df_temp])
        df_temp.to_csv(f'E:\Data\Eve-Online\ITEMs_csv\item_{idx}.csv')
    # if out_dct is None:
    #     return out_df
    # out_dct[hash] = out_df

In [6]:
item_sample = df_wars['victim.items'].iloc[0]

In [26]:
FLAT_JS(df_wars,'whatever')
#df_items.to_csv('E:\Data\Eve-Online\ITEMS.csv',index=False)

100%|██████████| 837851/837851 [35:10<00:00, 396.94it/s]  


In [2]:
file_names = os.listdir('E:\Data\Eve-Online\ITEMs_csv')

In [ ]:
df_ITEMS = pd.DataFrame()
for name in tqdm.tqdm(file_names):
    path = f'E:\Data\Eve-Online\ITEMs_csv\{name}'
    df_ = pd.read_csv(path,index_col=0)
    df_ITEMS = pd.concat([df_ITEMS,df_])

In [21]:
file_names_rest = file_names[274924:]

In [22]:
#Outer Epoch
shreshold = 5000
n_file = 0
while True:
    counter = 0 
    df_ITEMS = pd.DataFrame()
    finished = True
    for idx in tqdm.trange(len(file_names_rest)):
        name = file_names_rest[idx]
        counter+=1 
        path = f'E:\Data\Eve-Online\ITEMs_csv\{name}'
        df_ = pd.read_csv(path,index_col=0)
        df_ITEMS = pd.concat([df_ITEMS,df_])
        if counter > shreshold:
            finished = False 
            break 
    n_file += 1 
    df_ITEMS.to_csv(f'E:\Data\Eve-Online\ITEMS_Agg\item_agg_{n_file}.csv')
    if finished:
        break 
    file_names_rest = file_names_rest[idx:]
        

100%|██████████| 2927/2927 [00:05<00:00, 518.74it/s]


In [23]:
#Inner Epoch
file_names_2 = os.listdir('E:\Data\Eve-Online\ITEMS_Agg')
df_ITEMS_final = pd.DataFrame()
for name in tqdm.tqdm(file_names_2):
    path = f'E:\Data\Eve-Online\ITEMS_Agg\{name}'
    df_ = pd.read_csv(path,index_col=0)
    df_ITEMS_final = pd.concat([df_ITEMS_final,df_])

  0%|          | 0/114 [00:00<?, ?it/s]C:\Users\18066\AppData\Local\Temp\ipykernel_31992\2034501931.py:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ = pd.read_csv(path,index_col=0)
 46%|████▌     | 52/114 [00:10<00:11,  5.24it/s]C:\Users\18066\AppData\Local\Temp\ipykernel_31992\2034501931.py:6: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ = pd.read_csv(path,index_col=0)
 53%|█████▎    | 60/114 [00:12<00:12,  4.26it/s]C:\Users\18066\AppData\Local\Temp\ipykernel_31992\2034501931.py:6: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ = pd.read_csv(path,index_col=0)
 55%|█████▌    | 63/114 [00:12<00:10,  4.64it/s]C:\Users\18066\AppData\Local\Temp\ipykernel_31992\2034501931.py:6: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ = pd.read_csv(path,index_col=

In [37]:
df_ITEMS_final = df_ITEMS_final.drop(columns=['items'],axis=1)

In [43]:
df_ITEMS_final.reset_index(drop=True).to_csv('E:\Data\Eve-Online\ITEMS_Agg_All.csv')

In [34]:
1*False

0

In [5]:
name

'item_347429.csv'

In [6]:
df_ITEMS.to_csv('E:\Data\Eve-Online\ITEMS.csv',index=False)

In [4]:
df_sample = df_wars.head(1000)
out_dct = dict()
n_jobs = 4

threads = []
hash_id = 0
for df_slice in df_slicer(df_sample,n_jobs):
    hash_id +=1 
    threads.append(Process(target=FLAT_JS, args=(df_slice,hash_id)))

In [50]:
#Fianlly,remove item & attacker from war data
df_wars = df_wars.drop(['attackers','victim.items'],axis=1).reset_index(drop=True)
df_wars.to_csv('E:\\Data\\Eve-Online\\WARS.csv',index=False)

### Feature Eng (attackers):
1. Num of attackers 
2. attacker.corporation_id
3. attacker.final_blow_attacker_id
4. attacker.total_damage_delt 
5. attacker.average_security_status